The following section is used to extract data from the following Wikipedia page: <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"><strong>List of postal codes of Canada: M</strong></a>

import required libraries

In [1]:
import pandas as pd
import numpy as np
import folium
import ssl
import certifi
from urllib import request

Load the data and read into a pandas DataFrame structure

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

context = ssl._create_unverified_context()
response = request.urlopen(url, context = context)
html = response.read()

toronto_neighborhoods = pd.read_html(html, header = 0)[0]
toronto_neighborhoods.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Drop the rows that do not have assigned boroughs

In [3]:
toronto_neighborhoods.drop(toronto_neighborhoods[toronto_neighborhoods['Borough'] == 'Not assigned'].index, inplace = True)
toronto_neighborhoods.reset_index(inplace = True, drop = True)
toronto_neighborhoods.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Check which cell has a Borough but a Not assigned Neighbourhood

In [4]:
toronto_neighborhoods[toronto_neighborhoods.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood
6,M7A,Queen's Park,Not assigned


Use name of the Borough for the Neighbourhood that has 'Not assigned' value

In [5]:
empty_index = toronto_neighborhoods[toronto_neighborhoods.Neighbourhood == 'Not assigned'].index
toronto_neighborhoods.loc[empty_index,'Neighbourhood'] = toronto_neighborhoods.loc[empty_index,'Borough']
toronto_neighborhoods[toronto_neighborhoods.Neighbourhood == 'Not assigned']
toronto_neighborhoods.shape

(211, 3)

Next, we want to combine the Neighourboods that share the same Postcode. To do this, we first need to find out which Postcodes are shared by more than one Neighbourhood

In [6]:
postcode_count = toronto_neighborhoods.groupby('Postcode').count()
postcode_count = postcode_count[postcode_count['Borough'] != 1]
duplicated_postcodes = postcode_count.index
duplicated_postcodes
print('The number of duplicated postocdes are: {}'.format(len(duplicated_postcodes)))

The number of duplicated postocdes are: 57


Create a new dataframe that excludes those duplicated postcodes found in the previous step

In [7]:
single_neighborhoods = toronto_neighborhoods.set_index('Postcode')
single_neighborhoods = single_neighborhoods.drop(duplicated_postcodes, axis = 0)
single_neighborhoods.reset_index(inplace = True)
single_neighborhoods.head()
single_neighborhoods.shape

(46, 3)

Next, we add Boroughs that contain more than one Neighbourhood. This is done by looping through and creating dataframe for each duplicated postcode. The respective Postcode, Borough, Neighbourhood information can then be appended to the single_neighborhoods dataframe created in the previous step.

In [8]:
for postcode in duplicated_postcodes:
    temp_df = toronto_neighborhoods[toronto_neighborhoods.Postcode == postcode].reset_index(drop = True)
    temp_postcode = temp_df['Postcode'][0]
    temp_borough = temp_df['Borough'][0]
    temp_neighbourhood = temp_df['Neighbourhood'].str.cat(sep = ', ')
    
    single_neighborhoods = single_neighborhoods.append({'Postcode': temp_postcode, 'Borough': temp_borough, 'Neighbourhood': temp_neighbourhood}, ignore_index= True)
    
single_neighborhoods.tail()

,Postcode,Borough,Neighbourhood
98,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."
99,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."
100,M9M,North York,"Emery, Humberlea"
101,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
102,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


Rename the dataframe and get its dimension

In [9]:
simple_neighourhoods = single_neighborhoods
simple_neighourhoods.shape

(103, 3)

Obtain data from a csv file that has the geographical coordinates of each postal code

In [23]:
geo_data = pd.read_csv('Geospatial_Coordinates.csv')
geo_data.head()
# print('This csv file contains {} geographical coordiantes.'.format(geo_data.shape[0]))

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Make sure the column names are consistent in both tables

In [26]:
geo_data.rename(columns={'Postal Code':'Postcode'}, inplace = True)
geo_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two tables together

In [27]:
merged_df = pd.merge(single_neighborhoods, geo_data, how = 'outer')
merged_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
3,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
4,M3B,North York,Don Mills North,43.745906,-79.352188
5,M6B,North York,Glencairn,43.709577,-79.445073
6,M4C,East York,Woodbine Heights,43.695344,-79.318389
7,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
8,M6C,York,Humewood-Cedarvale,43.693781,-79.428191
9,M4E,East Toronto,The Beaches,43.676357,-79.293031
